In [1]:
from collections import defaultdict
from itertools import chain
import networkx as nx

In [2]:
def read_movement_instructions(instruction_row: str) -> list[tuple[str, int]]:
    instruction_row = instruction_row.rstrip()
    if instruction_row[-1] in ["R", "L"]:
        instruction_row = instruction_row + "0"
    if instruction_row[0] not in ["R", "L"]:
        instruction_row = "R" + instruction_row
    char_lst = []
    int_lst = []
    last_char = None
    for idx, entry in enumerate(instruction_row):
        if entry in ["L", "R"]:
            char_lst.append(entry)
            if last_char is not None:
                int_lst.append(int(instruction_row[last_char + 1 : idx]))
            last_char = idx
    int_lst.append(int(instruction_row[last_char + 1 :]))

    assert len(char_lst) == len(int_lst)
    return list(zip(char_lst, int_lst))

In [3]:
def parse_input(filename: str) -> tuple[nx.Graph, list[tuple[str, int]]]:
    graph = nx.Graph()

    def _add_neighbor(u_idx: tuple[int, int], v_idx: tuple[int, int], key: str) -> bool:
        if not graph.has_node(v_idx) or not graph.has_node(u_idx):
            return False

        if graph.nodes[u_idx]["passable"] and graph.nodes[v_idx]["passable"]:
            graph.add_edge(u_idx, v_idx, direction=key)
        return True

    boundary_left = defaultdict(list)
    boundary_top = defaultdict(list)
    with open(filename) as f:
        for row_idx, row in enumerate(f):
            if not row.rstrip():
                for col_idx, col_lst in boundary_top.items():
                    for lowest_row_idx in range(row_idx - 1, -1, -1):
                        if graph.has_node((lowest_row_idx, col_idx)):
                            _add_neighbor(
                                (lowest_row_idx, col_idx),
                                col_lst[-1],
                                "vertical",
                            )
                            break

                movement_instructions = read_movement_instructions(next(f))
                break

            for col_idx, char in enumerate(row):
                if char == " " or char == "\n":
                    if boundary_top[col_idx]:
                        _add_neighbor(
                            (row_idx - 1, col_idx),
                            boundary_top[col_idx][-1],
                            "vertical",
                        )

                    if boundary_left[row_idx]:
                        _add_neighbor(
                            (row_idx, col_idx - 1),
                            boundary_left[row_idx][-1],
                            "horizontal",
                        )
                    continue

                graph.add_node(
                    (row_idx, col_idx),
                    passable=(char != "#"),
                )

                if not _add_neighbor(
                    (row_idx, col_idx), (row_idx, col_idx - 1), "horizontal"
                ):
                    boundary_left[row_idx].append((row_idx, col_idx))
                if not _add_neighbor(
                    (row_idx, col_idx), (row_idx - 1, col_idx), "vertical"
                ):
                    boundary_top[col_idx].append((row_idx, col_idx))

        return graph, movement_instructions

In [4]:
def starting_pos(graph: nx.Graph) -> tuple[tuple[int, int], int]:
    col_idx = 0
    while True:
        if graph.has_node((0, col_idx)):
            if graph.nodes[(0, col_idx)]["passable"]:
                return ((0, col_idx), 3)
        else:
            col_idx += 1

In [5]:
def graph_step(graph: nx.Graph, pos: tuple[int, int], dir: int) -> tuple[int, int]:
    dir_strs = {
        0: "horizontal",
        1: "vertical",
        2: "horizontal",
        3: "vertical",
    }
    row_steps = {
        0: 0,
        1: 1,
        2: 0,
        3: -1,
    }
    col_steps = {
        0: 1,
        1: 0,
        2: -1,
        3: 0,
    }

    new_pos = (pos[0] + row_steps[dir], pos[1] + col_steps[dir])

    if graph.has_edge(pos, new_pos):
        return new_pos

    if graph.has_node(new_pos):
        return pos

    # we have to wrap
    for u_idx, v_idx, direction in graph.edges(pos, "direction"):
        if direction != dir_strs[dir]:
            continue
        if abs(u_idx[0] - v_idx[0]) + abs(u_idx[1] - v_idx[1]) > 1:
            ret_pos = v_idx if u_idx == pos else u_idx
            return ret_pos
    return pos

In [6]:
def task_one(
    graph: nx.Graph, movement_instructions: list[tuple[str, int]], verbose: bool = False
) -> tuple[int, nx.Graph, list[tuple[tuple[int, int], int]]]:
    pos, dir = starting_pos(graph)
    path_lst = []
    for instr, amount in movement_instructions:
        if instr == "L":
            dir = (dir - 1) % 4
        elif instr == "R":
            dir = (dir + 1) % 4
        else:
            raise AssertionError(dir)
        if verbose:
            print(f"Instruction {instr} {amount}: Now at {pos} facing {dir}")

        for _step in range(amount):
            new_pos = graph_step(graph, pos, dir)
            if new_pos != pos:
                path_lst.append((pos, dir))
                if verbose:
                    print(f"\tStep {_step}: from {pos} to {new_pos}")
            pos = new_pos

    path_lst.append((pos, dir))
    return 1000 * (pos[0] + 1) + 4 * (pos[1] + 1) + dir, graph, path_lst

In [7]:
val, graph, path_lst = task_one(*parse_input("test-input.txt"))
print(val, path_lst[-1])

6032 ((5, 7), 0)


In [8]:
val, graph, path_lst = task_one(*parse_input("input.txt"))
print(val, path_lst[-1])

43466 ((42, 115), 2)


In [9]:
def print_maze(graph: nx.Graph, path_lst: list[tuple[tuple[int, int], int]]):
    ret_str = defaultdict(str)
    next_lines = defaultdict(str)
    max_row_idx = max(node[0] for node in graph.nodes)
    max_col_idx = max(node[1] for node in graph.nodes)
    for row_idx in range(-1, max_row_idx + 1):
        for col_idx in range(-1, max_col_idx + 2):
            if graph.has_node((row_idx, col_idx)):
                ret_str[row_idx] += (
                    "o" if graph.nodes[(row_idx, col_idx)]["passable"] else "#"
                )
                if graph.has_edge((row_idx, col_idx), (row_idx, col_idx + 1)):
                    ret_str[row_idx] += "–"
                else:
                    ret_str[row_idx] += " "
                if graph.has_edge((row_idx, col_idx), (row_idx + 1, col_idx)):
                    next_lines[row_idx] += "| "
                elif not graph.has_node((row_idx + 1, col_idx)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx, col_idx), "direction")
                    if dir == "vertical" and (row_idx - 1, col_idx) not in [u, v]
                ]:
                    next_lines[row_idx] += "| "
                else:
                    next_lines[row_idx] += "  "
            else:
                if graph.has_node((row_idx, col_idx - 1)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx, col_idx - 1), "direction")
                    if dir == "horizontal" and (row_idx, col_idx - 2) not in [u, v]
                ]:
                    ret_str[row_idx] += "– "
                elif graph.has_node((row_idx, col_idx + 1)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx, col_idx + 1), "direction")
                    if dir == "horizontal" and (row_idx, col_idx + 2) not in [u, v]
                ]:
                    ret_str[row_idx] += " –"
                else:
                    ret_str[row_idx] += "  "

                if graph.has_node((row_idx + 1, col_idx)) and [
                    dir
                    for u, v, dir in graph.edges((row_idx + 1, col_idx), "direction")
                    if dir == "vertical" and (row_idx + 2, col_idx) not in [u, v]
                ]:
                    next_lines[row_idx] += "| "
                else:
                    next_lines[row_idx] += "  "

    dir_chars = {
        0: ">",
        1: "v",
        2: "<",
        3: "^",
    }

    for pos, dir in path_lst:
        tmp = ret_str[pos[0]] if dir in [0, 2] else next_lines[pos[0]]

        if dir == 0:
            ret_str[pos[0]] = (
                tmp[: 2 * pos[1] + 3] + dir_chars[dir] + tmp[3 + 2 * (pos[1]) + 1 :]
            )
        if dir == 1:
            next_lines[pos[0]] = (
                tmp[: 2 * pos[1] + 2]
                + dir_chars[dir]
                + " "
                + tmp[2 + 2 * (pos[1] + 1) :]
            )
        elif dir == 2:
            ret_str[pos[0]] = (
                tmp[: 2 * pos[1] + 1] + dir_chars[dir] + tmp[1 + 2 * pos[1] + 1 :]
            )
        elif dir == 3:
            next_lines[pos[0] - 1] = (
                tmp[: 2 * pos[1] + 2]
                + dir_chars[dir]
                + " "
                + tmp[2 + 2 * (pos[1] + 1) :]
            )

    return "\n".join(
        chain.from_iterable(
            [ret_str[idx], next_lines[idx]] for idx in sorted(ret_str.keys())
        )
    )

In [10]:
graph, movement_instructions = parse_input("test-input.txt")
val, graph, path_lst = task_one(graph, movement_instructions)
print(print_maze(graph, path_lst))

                                    
                  | | |             
                  o>o>o #           
                  |   v             
                 –o # o–o –         
                      v |           
                  # o–o–o           
                    | v |           
                 –o–o–o–o –         
  | | |   | | | | | | v             
  o–o–o # o–o–o–o–o–o–o #           
  | | |   | | | v   | v             
 –o>o>o>o–o–o–o–o># o–o>o>–         
  | |   v | | |     | | |           
 –o–o # o–o–o–o # o–o–o–o –         
  | |   v | | |   | |   |           
 –o–o–o–o>o>o>o>o–o–o # o –         
  | | |   | | | v | |     | |   |   
                 –o–o–o # o–o–o–o – 
                  | | |   |   | |   
                 –o–o–o–o–o # o–o – 
                  |   | | |   | |   
                 –o # o–o–o–o–o–o – 
                  |   | | | |   |   
                 –o–o–o–o–o–o # o – 
                  | | |   | |   |   


# Task 2

In [11]:
def get_cube_face(row_idx: int, col_idx: int, face_size: int) -> int:
    row_section = row_idx // face_size
    col_section = col_idx // face_size

    face_dict = {
        (0, 1): 0,
        (0, 2): 1,
        (1, 1): 2,
        (2, 0): 3,
        (2, 1): 4,
        (3, 0): 5,
    }
    return face_dict.get((row_section, col_section), None)


def at_cube_boundary(row_idx: int, col_idx: int, face_size: int) -> tuple[bool, bool]:
    def _check_idx(idx: int) -> bool:
        return idx % face_size == 0 or idx % -face_size == -1

    return (_check_idx(row_idx), _check_idx(col_idx))


def direct_neighbor(
    row_idx: int, col_idx: int, dir: int
) -> tuple[tuple[int, int], int]:
    row_steps = {
        0: 0,
        1: 1,
        2: 0,
        3: -1,
    }
    col_steps = {
        0: 1,
        1: 0,
        2: -1,
        3: 0,
    }

    return (row_idx + row_steps[dir], col_idx + col_steps[dir]), dir


def get_cube_neighbor(
    row_idx: int, col_idx: int, dir: int, face_size: int = 50
) -> tuple[tuple[int, int], int]:
    if not any(at_cube_boundary(row_idx, col_idx, face_size)):
        return direct_neighbor(row_idx, col_idx, dir)

    if (dir in [0, 2] and not at_cube_boundary(row_idx, col_idx, face_size)[1]) or (
        dir in [1, 3] and not at_cube_boundary(row_idx, col_idx, face_size)[0]
    ):
        return direct_neighbor(row_idx, col_idx, dir)

    face_idx = get_cube_face(row_idx, col_idx, face_size)

    if face_idx == get_cube_face(
        *direct_neighbor(row_idx, col_idx, dir)[0], face_size=face_size
    ):
        return direct_neighbor(row_idx, col_idx, dir)

    row_section, col_section = row_idx // face_size, col_idx // face_size
    section_row, section_col = row_idx % face_size, col_idx % face_size

    def _construct_idx(
        row_sec_offset: int, col_sec_offset: int, row_offset: int, col_offset: int
    ) -> tuple[int, int]:
        return (face_size * (row_section + row_sec_offset)) + row_offset, (
            face_size * (col_section + col_sec_offset)
        ) + col_offset

    if dir == 0:  # right
        if face_idx in [0, 3]:
            return direct_neighbor(row_idx, col_idx, dir)
        elif face_idx in [1]:  # move to 4 (right, inverted)
            return _construct_idx(2, -1, face_size - section_row - 1, face_size - 1), 2
        elif face_idx in [2, 5]:  # move to 1, 4 (bottom)
            return _construct_idx(-1, 1, face_size - 1, section_row), 3
        elif face_idx in [4]:  # move to 1 (right, inverted)
            return _construct_idx(-2, 1, face_size - section_row - 1, face_size - 1), 2
    if dir == 1:  # down
        if face_idx in [0, 2, 3]:
            return direct_neighbor(row_idx, col_idx, dir)
        elif face_idx in [1, 4]:  # move to 2, 5 (right)
            return _construct_idx(1, -1, section_col, face_size - 1), 2
        elif face_idx in [5]:  # move to 1 (top)
            return _construct_idx(-3, 2, 0, section_col), 1
    elif dir == 2:  # left
        if face_idx in [0]:  # move to 3 (left, inverted)
            return _construct_idx(2, -1, face_size - section_row - 1, 0), 0
        elif face_idx in [1, 4]:
            return direct_neighbor(row_idx, col_idx, dir)
        elif face_idx in [2]:  # move to 3 (top)
            return _construct_idx(1, -1, 0, section_row), 1
        elif face_idx in [3]:  # move to 0 (left, inverted)
            return _construct_idx(-2, 1, face_size - section_row - 1, 0), 0
        elif face_idx in [5]:  # move to 0 (top)
            return _construct_idx(-3, 1, 0, section_row), 1
    if dir == 3:  # up
        if face_idx in [0]:  # move to 5 (left)
            return _construct_idx(3, -1, section_col, 0), 0
        if face_idx in [1]:  # move to 5 (bottom)
            return _construct_idx(3, -2, face_size - 1, section_col), 3
        if face_idx in [2, 4, 5]:
            return direct_neighbor(row_idx, col_idx, dir)
        elif face_idx in [3]:  # move to 2 (left)
            return _construct_idx(-1, 1, section_col, 0), 0

    raise ValueError

In [12]:
def test_cube_neighbor(face_size: int = 50):
    face_coords = {
        0: (0, 1),
        1: (0, 2),
        2: (1, 1),
        3: (2, 0),
        4: (2, 1),
        5: (3, 0),
    }

    def _test(test_vals):
        for input, output in test_vals.items():
            in_face, dir, in_seg_row_idx, in_seg_col_idx = input
            out_face, border_str, inverted = output
            input_row_idx = face_coords[in_face][0] * face_size + in_seg_row_idx
            input_col_idx = face_coords[in_face][1] * face_size + in_seg_col_idx

            output_row_idx = face_coords[out_face][0] * face_size
            output_col_idx = face_coords[out_face][1] * face_size

            def _inv(val):
                return face_size - 1 - val if inverted else val

            if border_str == "right":
                output_row_idx += _inv(st)
                output_col_idx += face_size - 1
                expected_new_dir = 2
            elif border_str == "bottom":
                output_col_idx += _inv(st)
                output_row_idx += face_size - 1
                expected_new_dir = 3
            elif border_str == "left":
                output_row_idx += _inv(st)
                expected_new_dir = 0
            elif border_str == "top":
                output_col_idx += _inv(st)
                expected_new_dir = 1
            else:
                raise AssertionError

            calculated_neighbor, new_dir = get_cube_neighbor(
                input_row_idx, input_col_idx, dir
            )
            assert (calculated_neighbor, new_dir) == (
                (
                    output_row_idx,
                    output_col_idx,
                ),
                expected_new_dir,
            ), f"{input} | {output}: {calculated_neighbor} != { (output_row_idx, output_col_idx)}"

    for st in [0, 10, face_size - 1]:
        test_vals = {
            # 0
            (0, 0, st, face_size - 1): (1, "left", False),
            (0, 1, face_size - 1, st): (2, "top", False),
            (0, 2, st, 0): (3, "left", True),
            (0, 3, 0, st): (5, "left", False),
            # 1
            (1, 0, st, face_size - 1): (4, "right", True),
            (1, 1, face_size - 1, st): (2, "right", False),
            (1, 2, st, 0): (0, "right", False),
            (1, 3, 0, st): (5, "bottom", False),
            # 2
            (2, 0, st, face_size - 1): (1, "bottom", False),
            (2, 1, face_size - 1, st): (4, "top", False),
            (2, 2, st, 0): (3, "top", False),
            (2, 3, 0, st): (0, "bottom", False),
            # 3
            (3, 0, st, face_size - 1): (4, "left", False),
            (3, 1, face_size - 1, st): (5, "top", False),
            (3, 2, st, 0): (0, "left", True),
            (3, 3, 0, st): (2, "left", False),
            # 4
            (4, 0, st, face_size - 1): (1, "right", True),
            (4, 1, face_size - 1, st): (5, "right", False),
            (4, 2, st, 0): (3, "right", False),
            (4, 3, 0, st): (2, "bottom", False),
            # 5
            (5, 0, st, face_size - 1): (4, "bottom", False),
            (5, 1, face_size - 1, st): (1, "top", False),
            (5, 2, st, 0): (0, "top", False),
            (5, 3, 0, st): (3, "bottom", False),
        }
        _test(test_vals)


test_cube_neighbor()

In [13]:
def add_neighbor(
    graph: nx.Graph,
    u_idx: tuple[int, int],
    v_idx: tuple[int, int],
    key: str,
    new_direction: int,
) -> bool:
    if not graph.has_node(v_idx) or not graph.has_node(u_idx):
        return False

    if graph.nodes[u_idx]["passable"] and graph.nodes[v_idx]["passable"]:
        graph.add_edge(u_idx, v_idx, direction=key, new_direction=new_direction)
    return True

In [14]:
def parse_input_2(
    filename: str, face_size: int = 50
) -> tuple[nx.DiGraph, list[tuple[str, int]]]:
    graph = nx.DiGraph()

    with open(filename) as f:
        for row_idx, row in enumerate(f):
            if not row.rstrip():
                movement_instructions = read_movement_instructions(next(f))
                break

            for col_idx, char in enumerate(row):
                if char == " " or char == "\n":
                    continue

                graph.add_node(
                    (row_idx, col_idx),
                    passable=(char != "#"),
                )

        for node in graph.nodes:
            row_idx, col_idx = node
            for dir in range(4):
                new_pos, new_dir = get_cube_neighbor(row_idx, col_idx, dir)
                assert add_neighbor(
                    graph,
                    node,
                    new_pos,
                    key=dir,
                    new_direction=new_dir,
                )
        return graph, movement_instructions

In [15]:
def task_two(
    graph: nx.DiGraph,
    movement_instructions: list[tuple[str, int]],
    verbose: bool = False,
) -> tuple[int, nx.DiGraph, list[tuple[tuple[int, int], int]]]:
    pos, dir = starting_pos(graph)
    path_lst = []

    def _graph_step(
        graph: nx.DiGraph, pos: tuple[int, int], dir: int
    ) -> tuple[tuple[int, int], int]:
        for _, v, data in graph.out_edges(pos, data=True):
            if data["direction"] == dir:
                return v, data["new_direction"]
        return pos, dir

    for instr, amount in movement_instructions:
        if instr == "L":
            dir = (dir - 1) % 4
        elif instr == "R":
            dir = (dir + 1) % 4
        else:
            raise AssertionError(dir)

        for _step in range(amount):
            new_pos, new_dir = _graph_step(graph, pos, dir)
            if new_pos == pos:
                break
            pos, dir = new_pos, new_dir
            path_lst.append((pos, dir))

    path_lst.append((pos, dir))
    return 1000 * (pos[0] + 1) + 4 * (pos[1] + 1) + dir, graph, path_lst

In [16]:
val, graph, path_lst = task_two(*parse_input_2("input.txt"))
assert val == 162155
print(val, path_lst[-1])

162155 ((161, 37), 3)
